# 推荐系统算法

这里简单介绍下推荐系统中最为主要的**协同过滤算法**，大致分为如下几类：

* 基于用户的协同过滤（给用户推荐与他相似的人购买的物品）
* 基于商品的协同过滤（给用户推荐和他之前喜欢的物品相似的物品）
* 基于模型的协同过滤：关联算法，聚类算法，分类算法，回归算法，矩阵分解，神经网络,图模型以及隐语义模型都属于这个范畴。

而本次实战使用的是矩阵分解算法。

矩阵分解其实是数学上的一个经典问题。大家从线性代数中可以知道，
**矩阵可以做SVD分解、Cholesky分解等，就好比任何大于1的正整数都可以分解成若干质数的乘积，矩阵分解可以认为是一种信息压缩。**

下图是一个用户电影评分矩阵。矩阵的每行表示一个用户，每列表示一部电影，矩阵中每个位置的值，代表某个用户对某个电影的评分值。

![img](img/1.png)

* R矩阵:用户对电影的评分组合矩阵，
* 用户矩阵，每一个被压缩的行向量代表一个用户的信息向量，
* 电影矩阵，每一个被压缩列向量代表一个电影的信息向量。

而这样的矩阵分解压缩过程，使得用户矩阵和电影矩阵都具有了一定的语义信息，必须强调的是**用户矩阵行向量的维数和电影矩阵列向量维数是相等的**。所以本质上就是将每个用户和每个电影通过已有的打分信息Embedding到同一维度的信息向量空间。

**接下来我们就学习一下如何使用keras对R矩阵进行矩阵分解，获得每个电影和每个用户的信息向量。**

In [5]:
import pandas as pd
import numpy as np

rating = pd.read_csv("data/ratings.dat",sep="::",header=None)
rating.columns = ['userId','movieId','rating','timestamp']
rating.head()

/Users/zhaoyadong/anaconda3/envs/nlp/lib/python3.6/site-packages/ipykernel_launcher.py:4: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  after removing the cwd from sys.path.


,userId,movieId,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [6]:
num_user = np.max(rating["userId"])
num_movie = np.max(rating["movieId"])
print(num_user,num_movie,len(rating))

6040 3952 1000209


其中num_user = 6040, num_movie = 3952 len(rating)=1000209。

意味着我的数据中有6040为观众，3952 部电影，得到了1000209个评分数据。

从这些我们可以计算出上图用户电影组合的R矩阵的填充率。

1000209/(6040*3952) = 0.04190220560634904

这说明只有4.2%的用户电影组合有评分，当然这和实际情况是相符的，毕竟一个人只会给很少部分的电影评分，所以我们发现用户对电影的评分组合矩阵R极其稀疏。

所以接下来我们要做的就是**预测那些没有评分的用户电影组合可能的得分，填充R矩阵，这样就可以为用户推荐模型预测得分较高的电影。**

# 模型搭建

In [9]:
from tensorflow.keras import Model
import tensorflow.keras.backend as K
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Reshape
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dot

K.clear_session()

def Recmand_model(num_user,num_movie,k):
    input_uer = Input(shape=[None,],dtype="int32")
    model_uer = Embedding(num_user+1,k,input_length = 1)(input_uer)
    model_uer = Reshape((k,))(model_uer)
    
    input_movie = Input(shape=[None,],dtype="int32")
    model_movie  = Embedding(num_movie+1,k,input_length = 1)(input_movie)
    model_movie = Reshape((k,))(model_movie)
    
    out = Dot(1)([model_uer,model_movie])
    model = Model(inputs=[input_uer,input_movie], outputs=out)
    model.compile(loss='mse', optimizer='Adam')
    model.summary()
    return model

这里就是**矩阵分解的部分**，模型的架构图如下图所示：

将用户和电影通过**Eembdding层压缩到k维度向量，**

然后简单粗暴直接向量点乘，

得到用户对电影的预测评分。

这里误差采用平方误差MSE，优化器采用的是Adam。

> 1.对于loss函数来说，是否应该加入正则项和用户&物品的偏置，评分系统的平均分？

> 2.这样寻找用户和物品的embedding的话是不是就是在做SVD分解？还是说利用深度学习的方法来让它自己找出来他们分解后的矩阵呢？

> 1.loss函数中的正则项我没有加，加了从原理上来说应该更好。

> 2.对的就是做类似于SVD的矩阵分解，只不过利用深度学习的方式找分解后分矩阵。

![img](img/2.png)

In [10]:
model = Recmand_model(num_user,num_movie,100)

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, None, 100)    604100      input_1[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 100)    395300      input_2[0][0]                    
______________________________________________________________________________________________

# 数据准备

将数据准备成( [用户ID, 电影ID] , 用户ID对电影ID的评分 ）这种格式。接下来就可以把数据喂给模型了。

In [12]:
train_user = rating["userId"].values
train_movie = rating["movieId"].values

train_x = [train_user,train_movie]
train_y = rating["rating"].values

拿到输入数据之后，设置好batch_size,epoch，就可以进行训练了。运行下面代码让模型跑起来。

# 模型训练

In [13]:
model.fit(train_x,train_y,batch_size = 100,epochs =10)

Train on 1000209 samples
Epoch 1/10
1000209/1000209 [==============================] - 36s 35us/sample - loss: 2.5846
Epoch 2/10
1000209/1000209 [==============================] - 36s 36us/sample - loss: 0.8020
Epoch 3/10
1000209/1000209 [==============================] - 38s 38us/sample - loss: 0.6974
Epoch 4/10
1000209/1000209 [==============================] - 40s 40us/sample - loss: 0.5919
Epoch 5/10
1000209/1000209 [==============================] - 38s 38us/sample - loss: 0.4942
Epoch 6/10
1000209/1000209 [==============================] - 38s 38us/sample - loss: 0.4199
Epoch 7/10
1000209/1000209 [==============================] - 39s 39us/sample - loss: 0.3704
Epoch 8/10
1000209/1000209 [==============================] - 39s 39us/sample - loss: 0.3369
Epoch 9/10
1000209/1000209 [==============================] - 39s 39us/sample - loss: 0.3137
Epoch 10/10
1000209/1000209 [==============================] - 38s 38us/sample - loss: 0.2972


十个epoch之后loss只有0.2972，这样我们就可以不严谨的下结论：模型的预测误差不超出0.1，接下来是预测部分。

# 模型预测

从之前读入数据中可以得知，**userId为1的用户，没有对movieId为2的电影评分。**

我们就用模型试试userId为1的用户会为movieId为2的电影打多少数分呢？运行下方代码，便能知晓。

In [26]:
rating.sort_values(['userId','movieId'],ascending=True).head()

,userId,movieId,rating,timestamp
40,1,1,5,978824268
25,1,48,5,978824351
39,1,150,5,978301777
44,1,260,4,978300760
23,1,527,5,978824195


In [27]:
model.predict([[1],[2]])

array([[2.417516]], dtype=float32)

输出结果：**array([[2.417516]], dtype=float32)**

模型预测为2.4，而评分的总分为5分，意味着userId为1的用户很有可能会喜欢movieId为2的电影。

可以考虑将movieId为2的电影推荐给userId为1的用户。

# 总结

这里只是采用了最简单的方式做了一个简单的推荐系统，而且此方式很难解决新的电影和新的用户的推荐问题。推荐系统是门很深的学问，算法不仅需要考虑到推荐的准确率，覆盖率，还要考虑到推荐内容的丰富性和新颖性。人是很容易改变和厌倦的动物，所以，笔者有时候在想真会出现一个一直都懂你的推荐算法吗？